## Outcome prediction after Chikungunya hospitalization

#### MC853 - Unicamp

- Leandro Henrique Silva Resende – 213437 

- Pietro Grazzioli Golfeto – 223694 

- Yvens Ian Prado Porto – 184031 

In [77]:
# Required Libraries
# We used Python 3.10.12
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import (StratifiedKFold)
from sklearn.metrics import precision_score
import time
from sklearn.metrics import (roc_auc_score,           # For evaluating model performance
                             recall_score)   
from sklearn.svm import SVC                           # For Support Vector Classifier
from sklearn.linear_model import LogisticRegression   # For Logistic Regression Classifier
from sklearn.ensemble import (RandomForestClassifier, # For ensemble classifiers
                              GradientBoostingClassifier,
                              BaggingClassifier)
from sklearn.neural_network import MLPClassifier      # For Multi-layer Perceptron Classifier
from sklearn.neighbors import KNeighborsClassifier  # For K-Nearest Neighbors Classifier
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [78]:
# Paths to the data (change according to your system)
leandro_path = {
    'X_train_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/X_train.csv',
    'y_train_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/y_train.csv',
    'X_test_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/X_test.csv',
    'y_test_path': '/home/leandro/Documentos/UNICAMP/MC853/DataSUS-Chikungunya-ML/datasets/y_test.csv',
}

pietro_path = {
    'X_train_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/X_train.csv',
    'y_train_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/y_train.csv',
    'X_test_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/X_test.csv',
    'y_test_path': '/home/pietro/Desktop/DataSUS-Chikungunya-ML/datasets/y_test.csv',
}

In [79]:
# Set the path based on the user
if os.path.isfile(leandro_path['X_train_path']):
    path = leandro_path
elif os.path.isfile(pietro_path['X_train_path']):
    path = pietro_path
else:
    raise Exception('Path not found. Please check the paths in the script.')

# Get CSV files path (modify to match your file path)
X_train_path = os.path.expanduser(path['X_train_path'])
y_train_path = os.path.expanduser(path['y_train_path'])
X_test_path = os.path.expanduser(path['X_test_path'])
y_test_path = os.path.expanduser(path['y_test_path'])

In [80]:
X_train = pd.read_csv(X_train_path, low_memory=False)
y_train = pd.read_csv(y_train_path, low_memory=False)
X_test = pd.read_csv(X_test_path, low_memory=False)
y_test = pd.read_csv(y_test_path, low_memory=False)

In [81]:
X_train.head()

,AGE,GENDER,PREGNANT,FEBRE,MIALGIA,CEFALEIA,EXANTEMA,VOMITO,NAUSEA,DOR_COSTAS,...,REGION_MIDWEST,REGION_SOUTHEAST,REGION_SOUTH,TIME_DIFF_DAYS,TIME,WHITE,BLACK,YELLOW,BROWN,INDIGENOUS
0,10.0,0,0.0,1,1,1,1,0,1,1,...,0,0,0,1,26,0,0,0,1,0
1,29.0,1,0.0,1,1,1,1,0,1,1,...,0,0,0,5,25,0,0,0,1,0
2,11.0,1,0.0,1,1,1,0,0,0,0,...,0,0,0,2,37,0,0,0,1,0
3,5.0,1,0.0,1,0,1,0,0,1,0,...,0,0,0,31,38,0,0,0,1,0
4,11.0,1,0.0,1,1,1,0,0,0,0,...,0,0,0,3,32,0,0,0,1,0


In [82]:
X_train.columns

Index(['AGE', 'GENDER', 'PREGNANT', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA',
       'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA',
       'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES',
       'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT',
       'AUTO_IMUNE', 'CONFIRMED_CASE', 'CRITERIO', 'REGION_NORTH',
       'REGION_NORTHEAST', 'REGION_MIDWEST', 'REGION_SOUTHEAST',
       'REGION_SOUTH', 'TIME_DIFF_DAYS', 'TIME', 'WHITE', 'BLACK', 'YELLOW',
       'BROWN', 'INDIGENOUS'],
      dtype='object')

In [83]:
# Remove rows where TIME_DIFF_DAYS > 60 -> consider as outlier, before normalization
scaler = RobustScaler()
X_train[['AGE', 'TIME_DIFF_DAYS', 'TIME']] = scaler.fit_transform(X_train[['AGE', 'TIME_DIFF_DAYS', 'TIME']])

In [84]:
X_train[['AGE', 'TIME_DIFF_DAYS', 'TIME']].describe()

,AGE,TIME_DIFF_DAYS,TIME
count,21726.000000,21726.000000,21726.000000
mean,0.052730,0.572770,0.053436
std,0.612048,2.019177,0.522004
min,-0.794872,-0.750000,-0.756345
25%,-0.487179,-0.375000,-0.426396
50%,0.000000,0.000000,0.000000
75%,0.512821,0.625000,0.573604
max,2.256410,21.750000,0.832487


In [85]:
X_train[['AGE', 'TIME_DIFF_DAYS', 'TIME']].head()

,AGE,TIME_DIFF_DAYS,TIME
0,-0.538462,-0.625,-0.624365
1,-0.051282,-0.125,-0.629442
2,-0.512821,-0.500,-0.568528
3,-0.666667,3.125,-0.563452
4,-0.512821,-0.375,-0.593909


In [86]:
# # Drop CRITERIO column (not relevant with CONFIRMED_CASE)
X_train['CRITERIO'].value_counts()
X_train = X_train.drop(columns=['CRITERIO'])
X_test = X_test.drop(columns=['CRITERIO'])

In [87]:
def impute_missing(data_train, data_test, n_neighbors=3):
    """
    Impute missing values using the K-nearest neighbors algorithm.

    Parameters:
        data (pd.DataFrame): Input DataFrame with missing values.
        n_neighbors (int, optional): Number of neighbors to use for imputation. Defaults to 3.

    Returns:
        pd.DataFrame: DataFrame with missing values imputed using KNN.
    """
    # Initialize KNNImputer with the specified number of neighbors
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # Perform imputation
    imputed_data_train = imputer.fit_transform(data_train)
    imputed_data_test = imputer.transform(data_test)

    # Convert the imputed array back to a DataFrame
    imputed_df_train = pd.DataFrame(imputed_data_train, columns=data_train.columns, index=data_train.index)
    imputed_df_test = pd.DataFrame(imputed_data_test, columns=data_test.columns, index=data_test.index)

    return imputed_df_train, imputed_df_test


In [88]:
X_train, X_test = impute_missing(X_train, X_test, n_neighbors=3)

In [89]:
pipeline = Pipeline(steps=[
    ('o', SMOTE(sampling_strategy=0.4, random_state=42)),  # Increase minority to 40% of majority
    ('u', RandomUnderSampler(sampling_strategy=1.0, random_state=42))  # Then balance both classes equally
])

X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train, y_train)

In [90]:
print(y_train.sum())
print(y_train_balanced.sum())
print(y_train_balanced.value_counts())

EVOLUCAO    1586
dtype: int64
EVOLUCAO    8056
dtype: int64
EVOLUCAO
0           8056
1           8056
Name: count, dtype: int64


In [91]:
# Set the number of folds for cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [92]:
# Define a dictionary containing various classification algorithms

algorithms = {    
    'random_forest': RandomForestClassifier(n_estimators=100, max_depth=50, min_samples_split=2, random_state=0, n_jobs=-1),
    # Random Forest Classifier
    
    'logistic_regression': LogisticRegression(class_weight='balanced', random_state=0, n_jobs=-1),
    # Logistic Regression Classifier

    'knn': KNeighborsClassifier(n_neighbors=5, p=2, weights='uniform', n_jobs=-1),
    # K-Nearest Neighbors Classifier    
}

In [93]:
def evaluate_cv(X_train, y_train):
    '''
    Receives data to be evaluated and returns the average performance inside cross-validation, using 3 metrics.
    Applies over-under sampling to get balanced datasets and standardizes features.
    
    Parameters:
    data : DataFrame
        The dataset containing features and the target variable.
    
    Returns:
    df : DataFrame
        A DataFrame containing the mean and standard deviation of each algorithm's performance across 5-fold cross-validation.
        The performance metrics include AUC (mean and standard deviation), sensitivity (mean and standard deviation),
        specificity (mean and standard deviation), prec_n (mean and standard deviation), and prec_p (mean and standard deviation).
    '''
    # Record the start time
    start_time = time.time()
    
    # Initialize dictionaries to store metrics for each algorithm
    sen = {}
    spe = {}
    auc = {}
    prec_n = {}  # Negative precision
    prec_p = {}  # Positive precision
    
    for algorithm in algorithms.keys():
        sen[algorithm] = []
        spe[algorithm] = []
        auc[algorithm] = []
        prec_n[algorithm] = []
        prec_p[algorithm] = []

    # Iterate through each round of the cross-validation
    for train, test in kf.split(X_train, y_train):
        # Allocate train and test data
        X_train_fold, X_test_fold = X_train.iloc[train], X_train.iloc[test]
        y_train_fold, y_test_fold = y_train.iloc[train], y_train.iloc[test]

        # # Apply over-under sampling
        # X_train, y_train = data_sample(X_train, y_train)
            
        # X_train = imputer.fit_transform(X_train)
        # X_test = imputer.transform(X_test)
                
        # # Standardize features
        # X_train = preprocessing.fit_transform(X_train)
        # X_test = preprocessing.transform(X_test)

        # Iterate through each algorithm
        for algorithm, (clf) in algorithms.items():
            
            clf.fit((X_train_fold), y_train_fold)

            # Make predictions for the test data
            y_pred = clf.predict(X_test_fold)

            # Calculate sensitivity and specificity 
            recallscore = recall_score(y_test_fold, y_pred, labels=[0, 1], average=None)
            sen[algorithm].append(recallscore[1])
            spe[algorithm].append(recallscore[0])

            # Calculate precision for each class
            prec_score = precision_score(y_test_fold, y_pred, labels=[0, 1], average=None)
            prec_n[algorithm].append(prec_score[0])
            prec_p[algorithm].append(prec_score[1])

            # Calculate the area under the ROC curve
            aucscore = roc_auc_score(y_test_fold, (clf.predict_proba((X_test_fold)))[:, 1])     
            auc[algorithm].append(aucscore)

    # Create a DataFrame with the mean and standard deviation of each algorithm's performance across 5 folds 
    df = pd.DataFrame(columns=list(algorithms.keys()))

    # Map AUC metrics to the three algorithms
    df.loc['auc (mean)']  = [np.mean(auc['random_forest']), np.mean(auc['logistic_regression']), np.mean(auc['knn'])]
    df.loc['auc (stdev)'] = [np.std(auc['random_forest']),  np.std(auc['logistic_regression']),  np.std(auc['knn'])]

    # Map sensitivity (rcl_1) metrics to the three algorithms
    df.loc['rcl_1 (mean)']  = [np.mean(sen['random_forest']), np.mean(sen['logistic_regression']), np.mean(sen['knn'])]
    df.loc['rcl_1 (stdev)'] = [np.std(sen['random_forest']),  np.std(sen['logistic_regression']),  np.std(sen['knn'])]

    # Map specificity (rcl_0) metrics to the three algorithms
    df.loc['rcl_0 (mean)']  = [np.mean(spe['random_forest']), np.mean(spe['logistic_regression']), np.mean(spe['knn'])]
    df.loc['rcl_0 (stdev)'] = [np.std(spe['random_forest']),  np.std(spe['logistic_regression']),  np.std(spe['knn'])]

    # Map positive precision (prc_1) metrics to the three algorithms
    df.loc['prc_1 (mean)']  = [np.mean(prec_p['random_forest']), np.mean(prec_p['logistic_regression']), np.mean(prec_p['knn'])]
    df.loc['prc_1 (stdev)'] = [np.std(prec_p['random_forest']),  np.std(prec_p['logistic_regression']),  np.std(prec_p['knn'])]

    # Map negative precision (prc_0) metrics to the three algorithms
    df.loc['prc_0 (mean)']  = [np.mean(prec_n['random_forest']), np.mean(prec_n['logistic_regression']), np.mean(prec_n['knn'])]
    df.loc['prc_0 (stdev)'] = [np.std(prec_n['random_forest']),  np.std(prec_n['logistic_regression']),  np.std(prec_n['knn'])]

    # Record the end time
    end_time = time.time()

    # Calculate the time taken
    total_time = end_time - start_time

    # Display the DataFrame
    display(df)

    # Print the total time taken to run cross-validation
    print(f"Total time taken to run cross-validation: {total_time:.2f} seconds")

    return df

In [94]:
evaluate_cv(X_train_balanced, y_train_balanced)

,random_forest,logistic_regression,knn
auc (mean),0.968905,0.767606,0.911657
auc (stdev),0.002245,0.005914,0.003453
rcl_1 (mean),0.883192,0.711891,0.975795
rcl_1 (stdev),0.008014,0.014964,0.003396
rcl_0 (mean),0.944886,0.690665,0.638282
rcl_0 (stdev),0.002519,0.006896,0.008449
prc_1 (mean),0.941270,0.697086,0.729589
prc_1 (stdev),0.002434,0.003828,0.004382
prc_0 (mean),0.890033,0.705817,0.963504
prc_0 (stdev),0.006568,0.009431,0.004801


Total time taken to run cross-validation: 6.28 seconds


,random_forest,logistic_regression,knn
auc (mean),0.968905,0.767606,0.911657
auc (stdev),0.002245,0.005914,0.003453
rcl_1 (mean),0.883192,0.711891,0.975795
rcl_1 (stdev),0.008014,0.014964,0.003396
rcl_0 (mean),0.944886,0.690665,0.638282
rcl_0 (stdev),0.002519,0.006896,0.008449
prc_1 (mean),0.941270,0.697086,0.729589
prc_1 (stdev),0.002434,0.003828,0.004382
prc_0 (mean),0.890033,0.705817,0.963504
prc_0 (stdev),0.006568,0.009431,0.004801
